In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from final_project.loader import get_df

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def plot_freq(title, data):
    
    fig, ax = plt.subplots(figsize=(6,3))
    bars = ax.bar(['delayed' if x==1 else 'on time' for x in data[0]], data[1])

    ax.set_title(f'Class Instances in {title.capitalize()} Dataset')
    ax.set_xlabel('Count')
    ax.set_ylabel('Labels')

    print(data[1])
    ax.bar_label(bars, data[1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Show the chart
    plt.show()

In [ ]:
master_df = get_df(file="Flights_2018_1.csv")

In [ ]:
'''
process data for training, split into test/train/validation
'''
X = master_df.drop(columns=["ArrDel15","Duplicate"])
y = master_df[["ArrDel15"]]
y = y.ArrDel15.ravel() # flatten

print("# samples:", y.shape[0])

# split into train and test/validation (which is split in next line)
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=150)

# create validation and test sets each 15% of total data
X_test, X_validation, y_test, y_validation = train_test_split(X_test_val, y_test_val,
                                                    test_size=0.5,
                                                    random_state=150)
d = {"train": (X_train,y_train), "test":(X_test,y_test), "validation": (X_validation,y_validation)}


# number of classes, number of instances in each class
for each in d.keys():
    print(f"{each}:")
    print(" - Number of features: ", len(d[each][0].columns))
    print(" - Number of samples: ", len(d[each][0]))
    unique, counts = np.unique(d[each][1], return_counts=True)
    plot_freq(each, (unique, counts))
    print()


In [ ]:
# create train numpy arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
X_validation = X_validation.to_numpy()

# convert to tensors
X_train, y_train, X_test, y_test, X_validation, y_validation = map(
    torch.tensor, (X_train, y_train, X_test, y_test, X_validation, y_validation)
)

In [ ]:
# create model including fit and score methods
class FeedForward(nn.Module):
    def __init__(self, num_hidden_layers, num_nodes:int, num_features) -> None:
        super(FeedForward, self).__init__()
        self.num_classes = 2
        if num_hidden_layers == 1:
             self.layers = nn.Sequential(
                nn.Flatten(),
                nn.Linear(num_features, num_nodes),
                nn.ReLU(),
                nn.Linear(num_nodes, 1),
            )
        else:
            self.layers = nn.Sequential(
                nn.Flatten(),
                nn.Linear(num_features, num_nodes),
                nn.ReLU(),
                nn.Linear(num_nodes, num_nodes),
                nn.ReLU(),
                nn.Linear(num_nodes, 1),
            )

    def forward(self, xb):
        xb = xb.to(self.layers[1].weight.dtype)
        return self.layers(xb)
        
    # TODO add momentum
    def fit(self, train_dataset: TensorDataset, validation_dataset: TensorDataset, batch_size: int, 
            epochs: int, loss_function, learning_rate: float):
        
        # create dataloader for batching, shuffle to avoid overfitting/batch correlation
        train_dl = DataLoader(train_dataset, batch_size, shuffle=True)
        valid_dl = DataLoader(validation_dataset, batch_size, shuffle=True)

        # opt = torch.optim.SGD(self.parameters(), lr=learning_rate, momentum=.9, weight_decay=.0000001) # create optimizer TODO weight decay
        # TODO tune optimizer
        # opt = torch.optim.SGD(self.parameters(), lr=learning_rate) # create optimizer TODO weight decay
        opt = torch.optim.Adam(self.parameters(), lr=learning_rate)
        # store epoch losses
        training_losses = []
        validation_losses = []

        for epoch in tqdm(range(epochs)):
            self.train()
            epoch_training_loss = 0
            epoch_validation_loss = 0

            for xb, yb in train_dl:
                # run model on batch and get loss
                predictions = self(xb).squeeze()
                loss = loss_function(predictions, yb)
                # Back Propagation
                loss.backward()  # compute gradient
                opt.step()       # update weights
                opt.zero_grad()  # reset gradient

            self.eval()
            with torch.no_grad():
                for xb_tr, yb_tr in train_dl:
                    # train loss
                    train_predictions = self(xb_tr).squeeze()
                    training_loss = loss_function(train_predictions, yb_tr)
                    epoch_training_loss += (training_loss * xb_tr.shape[0])
                for xb_val, yb_val in valid_dl:
                    # val loss
                    val_predictions = self(xb_val).squeeze()
                    validation_loss = loss_function(val_predictions, yb_val) # get loss
                    epoch_validation_loss += (validation_loss * xb_val.shape[0])
                # get epoch loss
                num_train_samples = len(train_dataset)
                epoch_training_loss_normalized = epoch_training_loss / num_train_samples 
                training_losses.append(epoch_training_loss_normalized.item())

                num_val_samples = len(validation_dataset)
                epoch_validation_loss_normalized = epoch_validation_loss / (num_val_samples) 
                validation_losses.append(epoch_validation_loss_normalized.item())

        return training_losses, validation_losses

    def score(self, tensor_dataset: TensorDataset, batch_size=1) -> float:
        # reference: https://blog.paperspace.com/training-validation-and-accuracy-in-pytorch/
        self.eval()
        dataloader = DataLoader(tensor_dataset, batch_size, shuffle=True)
        all_predictions = torch.tensor([],dtype=torch.long)
        ground_truth_labels = torch.tensor([],dtype=torch.long)
        with torch.no_grad():
            for xb, yb in tqdm(dataloader):
                # run model on batch
                class_probabilities = self(xb)
                
                # choose most likely class for each sample
                predictions = (class_probabilities > 0.5).long().squeeze()
                
                # create running tensor with all true_label, predicted_label pairs 
                ground_truth_labels = torch.cat((ground_truth_labels, yb))
                all_predictions = torch.cat((all_predictions, predictions))
            
            # create a stack where the top layer is the ground truth, bottom is prediction
            true_pred_stack = torch.stack((ground_truth_labels, all_predictions))
            classifier_scores = precision_recall_fscore_support(ground_truth_labels, all_predictions)
            ConfusionMatrixDisplay.from_predictions(ground_truth_labels, all_predictions)
            confusion_mtx = confusion_matrix(ground_truth_labels, all_predictions)
            class_accuracy = {i:0 for i in range(self.num_classes)} # dictionary containing class accuracies
            for label in range(self.num_classes):
                # get all pairs with the same true label in the tensor stack
                class_pairs = list(filter(lambda pair: pair[0] == label, true_pred_stack.T)) 

                # calculate how many have correctly been predicted (true = predicted)
                num_correct = len(list(filter(lambda pair: pair[0] == pair[1], class_pairs)))

                # find accuracy
                class_accuracy[label] = num_correct/len(class_pairs)
                
            # get mean accuracy
            correct = sum(all_predictions == ground_truth_labels).item()
            mean_accuracy = correct / len(tensor_dataset)
            
            return mean_accuracy, class_accuracy, classifier_scores, confusion_mtx
                

In [ ]:
# create dataset and dataloader
train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)
valid_ds = TensorDataset(X_validation, y_validation)

In [ ]:
loss_function = nn.BCEWithLogitsLoss()
num_features = X_train.shape[1]
classes = y_train.unique()

In [ ]:
# exhaustive hyperparameter tuning based on the best final validation loss
def ffn_tune(num_hidden_layers, num_nodes):
    params = {"bs":(64,),
                "epoch":(50,),
                "learning_rate":(.01,)}
    # params = {"bs":(50, 64, 80),
    #             "epoch":(30,50),
    #             "learning_rate":(.008,.09)}
    # params = {"bs":(30,),
    #             "epoch":(50,),
    #             "learning_rate":(.01, .05, .09, .1, .3, .5)}
    best_model = {key:params[key] for key in params}
    best_model["best_loss"] = 100000000000
    for bs in params["bs"]:
        for epoch in params["epoch"]:
            for lr in params["learning_rate"]:
                # use validation loss
                model = FeedForward(num_hidden_layers, num_nodes, num_features)
                training_losses, valid_losses = model.fit(train_ds, valid_ds, bs, epoch, loss_function, lr)
                if valid_losses[-1] < best_model["best_loss"]:
                    best_model["model"]=model
                    best_model["best_loss"] = valid_losses[-1]
                    best_model["epoch"] = epoch
                    best_model["learning_rate"] = lr
                    best_model["bs"] = bs
                    best_model["valid_losses"] = valid_losses
                    best_model["training_losses"] = training_losses
                print("best loss: ",best_model["best_loss"])
    return best_model


In [ ]:
def ffn_evaluate(model):
# plot losses
    plt.plot(model["training_losses"], label="Training Loss")
    # print(model["training_losses"])
    plt.plot(model["valid_losses"], label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    # calculate accuracy
    d = {"train": train_ds, "test": test_ds, "validation": valid_ds}
    for dataset in d:
        print(f"Evaluating **{dataset}** dataset:")
        mean_accuracy, class_accuracy, classifier_scores, confusion_matrix = model["model"].score(d[dataset], model["bs"])
        print(f"Mean Accuracy: {mean_accuracy*100:.3f}")
        print(f"Mean per-class accuracy:")
        for key in class_accuracy:
            print(f"  {'delayed' if key==1 else 'on time'}{': '}{class_accuracy[key]*100:.3f}%")
        print(f"Precision: {classifier_scores[0]}")
        print(f"Recall: {classifier_scores[1]}")
        print(f"F-Beta Score: {classifier_scores[2]}")
        print(f"F1 Score: {classifier_scores}")
        print(confusion_matrix)
        print()

In [ ]:
# run model tuning and evaluation on the 4 combos of layers/nodes
def run_model(num_layers: int, num_nodes: int):
    print(f"{num_nodes} Nodes, {num_layers} Hidden Layer(s)")
    model = ffn_tune(num_layers,num_nodes)
    print("best batch size: ", model["bs"])
    print("best epoch: ", model["epoch"])
    print("best learning rate: ", model["learning_rate"])
    ffn_evaluate(model)

for pair in [(1,4)]:
    run_model(pair[0],pair[1])
